In [1]:
import numpy as np
import networkx as nx
from scipy.optimize import linprog
from math import comb

In [2]:
t = 3
q = t-1
V = 9 # TODO: try V = 8 ??
delta = 0.0125 # TODO: how far we can push delta to ??

In [1]:
def get_small_idx(all_graphs, t):
    for i,g in enumerate(all_graphs):
        if g.number_of_edges() > comb(t,2):
            return i
    return len(all_graphs)

In [3]:
ALLGRAPHS = sorted(nx.read_graph6("graph"+str(V)+".g6"), key = lambda G: G.number_of_edges()) # 274668 graphs on at most 9 vertices
ALLCC = np.array([np.count_nonzero([len(c)>1 for c in list(nx.connected_components(g))]) for g in ALLGRAPHS]) # get number of connected components of all graphs on at most 9 vertices
# ALLCC = np.array([len(list(nx.connected_components(G)))-1 for G in ALL9GRAPHS]) # TODO: isolated vertices as components??

In [4]:
ALLH = sorted(nx.read_graph6("H"+str(V)+".g6"), key = lambda G: G.number_of_edges())

In [5]:
def nodes(G):
    """
    return the number of nodes of graph G
    """
    counter=0
    for v in G.nodes():
        if G.degree(v) > 0: counter+=1
    return counter

def DC(x, q = 2, V = 9):
    """
    return the value of DC given x, q and V
    """
    print("query",x)
    for n in range(V+1,max(2*x+1,V+2)):
        print("n,V,x = ", n,V,x)
    return max([comb(n,V)/(q**(n-V)*comb(n-x,V-x)) for n in range(V+1,max(2*x+1,V+2))]) 

In [6]:
ISO_COUNT = np.load("iso_count"+str(V)+".npy")

In [7]:
DCs = np.array([DC(nodes(g), V = V) for g in ALLH]) # TODO: why truncating DC??

array([0.5       , 0.625     , 0.71428571, 0.83333333, 0.83333333,
       1.        , 0.71428571, 0.83333333, 0.29761905, 1.        ,
       0.29761905, 0.83333333, 1.        , 0.29761905, 0.83333333,
       0.5       , 0.5       , 0.5       , 1.        ])

In [8]:
small_index = get_small_idx(ALLGRAPHS, t)
mu_0 = 2**comb(t,2)-1
def check_condition_2(coefs):
    """
    check that all graphs with at most 3 edges have mu less than or equal to 1
    """
    sums = np.apply_along_axis(lambda x: np.sum(x*coefs), 1, ISO_COUNT[:small_index])
    return (np.max(np.abs(sums)[1:]) <=1) and sums[0] == mu_0

def check_condition_3(coefs):
    """
    check that all graphs with more than tC2 edges and at most V vertices have mu at most 1-delta
    check that the maximum bound is at most 1-delta by iterating over all graphs on at most V certices
    """
    small_sums = np.apply_along_axis(lambda x: np.sum(x*coefs), 1, ISO_COUNT[small_index:]) # graphs > tC2 edges and <= V vertices
    large_sums = np.apply_along_axis(lambda x: np.sum(x*coef_tilde(coefs)*DCs), 1, ISO_COUNT[1:]) # check graphs > tC2 edges and > V vertices
#     print("bound on small",np.max(np.abs(small_sums)),"bound on big",np.max(large_sums/q**(ALLCC[1:]-1)),"threshold", 1-delta)
    return (np.max(large_sums/q**(ALLCC[1:]-1)) <= 1 - delta) and (np.max(np.abs(small_sums)) <= 1 - delta)

In [ ]:
from multiprocessing import Pool, TimeoutError
def return_max_mu(coefs,indices):
    sums = np.apply_along_axis(lambda x:np.sum(x*coefs), 1, ISO_COUNT[indices[0]:indices[1]])
    return np.max(np.abs(sums))
def return_max_bound(coefs, indices):
    sums = np.apply_along_axis(lambda x:np.sum(x*coefs), 1, ISO_COUNT[indices[0]:indices[1]])
    return np.max(large_sums/q**(ALLCC[indices[0]:indices[1]]-1))


def check_condition_3_eff(coefs):
    """
    check that all graphs with more than tC2 edges and at most V vertices have mu at most 1-delta
    check that the maximum bound is at most 1-delta by iterating over all graphs on at most V certices
    """
    window = 10000
    index_list = list([i,min(i+window,len(ISO_COUNT))] for i in range(small_index,len(ISO_COUNT),window))
    with Pool(processes = 4) as pool:
        multiple_results = [pool.apply_async(return_max_mu, (i,)) for i in index_list]
        res = max([res.get(timeout=1) for res in multiple_results])<=1-delta
    
    index_list1 = list([i,min(i+window,len(ISO_COUNT))] for i in range(1,len(ISO_COUNT),window))
    with Pool(processes = 4) as pool:
        multiple_results1 = [pool.apply_async(return_max_bound, (i,)) for i in index_list1]
        res1 = max([res.get(timeout=1) for res in multiple_results1])<=1-delta
    return res and res1

In [ ]:
all_avail_coeff = []
# all disconnected H have 0 coefficients
for x3 in np.arange(-1.0,2.0,0.4):
    for x4 in np.arange(-2,5,1):
        for x7 in np.arange(-1.0,1.0,0.2):
            for x9 in np.arange(-1.0,1.0,0.4):
                for x12 in np.arange(-1,2,1):
                    for x14 in np.arange(-4.0,2.0,0.5):
                        for x18 in np.arange(-1.0,1.0,0.5):
                            coefs = np.array([7.0,-5,-1,x3,x4,0.0,0.0,x7,0.0,x9,0.0,0.0,x12,0.0,x14,0.0,0.0,0.0,x18])
                            res=check_condition_2(coefs)
                            res2 = check_condition_3(coefs)
                            if res and res2:
                                all_avail_coeff.append(coefs)
with open('coeff_res.npy', 'wb') as f:
    np.save(f,np.array(all_avail_coeff))